In [30]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew #for some statistics
import numpy as np
from sklearn.linear_model import LinearRegression

In [31]:
train_data = pd.read_csv('train.csv')

In [32]:
test_data = pd.read_csv('test.csv')

In [33]:
full = pd.concat([train_data,test_data], ignore_index=True)

In [34]:
#missing data
total_missing = train_data.isnull().sum().sort_values(ascending=False)
percent = (train_data.isnull().sum()/train_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total_missing, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(19)

,Total,Percent
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageCond,81,0.055479
GarageType,81,0.055479
GarageYrBlt,81,0.055479
GarageFinish,81,0.055479


#### Fill LotFrontage missing value

In [35]:
full['LotFrontage']=full.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

#### Then we filling in other missing values according to data_description.

#### Full data

In [36]:
cols=["MasVnrArea", "BsmtUnfSF", "TotalBsmtSF", "GarageCars", "BsmtFinSF2", "BsmtFinSF1", "GarageArea"]
for col in cols:
    full[col].fillna(0, inplace=True)
    
cols1 = ["PoolQC" , "MiscFeature", "Alley", "Fence", "FireplaceQu", "GarageQual", "GarageCond", "GarageFinish", "GarageYrBlt", "GarageType", "BsmtExposure", "BsmtCond", "BsmtQual", "BsmtFinType2", "BsmtFinType1", "MasVnrType"]
for col in cols1:
    full[col].fillna("None", inplace=True)
    
# fill in with mode
cols2 = ["MSZoning", "BsmtFullBath", "BsmtHalfBath", "Utilities", "Functional", "Electrical", "KitchenQual", "SaleType","Exterior1st", "Exterior2nd"]
for col in cols2:
    full[col].fillna(train_data[col].mode()[0], inplace=True)

In [37]:
def year_filiter(year):
    if year>=2000:
        label = 1
    elif year >= 1990:
        label =2
    elif year >= 1980:
        label =3
    elif year >= 1970:
        label =4
    elif year >= 1960:
        label =5
    else:
        label =6
    return label

In [38]:
def get_highly_skewed_col(_df,ratio_thres):
    result = {}
    for col in train_data.columns:
        max_ratio = (_df[col].value_counts()/_df[col].count()).max()
        if max_ratio > ratio_thres:
            result[col] = max_ratio
    return pd.Series(result)

In [39]:
full['YearBuiltlabel'] = full.YearBuilt.map(lambda x: year_filiter(x))

In [40]:
full['YearRemodAdd_label'] = full['YearRemodAdd'].map(lambda x: year_filiter(x))

#### Full data

#### Porch Summation: 

In [41]:
full['TotalPorch'] = full['OpenPorchSF'] + full['EnclosedPorch'] + full['ScreenPorch']

In [42]:
full.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [43]:
full.to_csv('Input_data.csv')

In [44]:
def get_unique_ratio(column):
    return len(column.unique())/len(column)

def is_date_or_time(column_name):
    date_keys = ["date","time"]
    for key in date_keys:
        if key in column_name.lower():
            return "date/time"
        
def get_data_type(column,unique_ratio_categorical, unique_ratio_ordinal):
    if is_date_or_time(column.name):
        return "date/time"
    else:
        if get_unique_ratio(column) < unique_ratio_categorical:
            return "categorical"
        else:
            if get_unique_ratio(column) > unique_ratio_ordinal:
                return "ordinal"
            else:
                if column.dtype == "object":
                    return "text"
                else:
                    return "numeric"
                
def variable_helper(input_df, unique_ratio_categorical, unique_ratio_ordinal):
    return {column:get_data_type(input_df[column], unique_ratio_categorical, unique_ratio_ordinal)\
            for column in input_df}

In [45]:
var_type = pd.DataFrame(pd.Series(variable_helper(full,0.02,0.8)),columns = ["col_type"])
types = pd.unique(var_type.col_type)

var_for_each_type = {_type: list(var_type[var_type.col_type==_type].index.values) for _type in types}
var_for_each_type['categorical']

['MSSubClass',
 'MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'LowQualFinSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageCars',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 '3SsnPorch',
 'PoolArea',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'MiscVal',
 'MoSold',
 'YrSold',
 'SaleType',
 'SaleCondition',
 'YearBuiltlabel',
 'YearRemodAdd_label']

In [46]:
drop_list = ['Street', 'Utilities', 'Condition2', 'RoofMatl', 'Heating',
             'LowQualFinSF', 'KitchenAbvGr', '3SsnPorch', 'PoolArea', 'PoolQC',
             'MiscFeature', 'MiscVal','Id', 'MSSubClass','LotAreaCut','Condition2','Alley', 
             'GarageYrBlt','BsmtFinSF1','BsmtFinSF2','BsmtFinType2','1stFlrSF', '2ndFlrSF',
             'TotRmsAbvGrd', 'GarageArea','Exterior1st','OpenPorchSF','EnclosedPorch','ScreenPorch',"YearBuilt","YearRemodAdd",
            'MoSold']

In [47]:
for col in drop_list:
    try:
        full =  full.drop(labels=col,axis=1)
    except:
        continue

In [48]:
full.columns

Index(['MSZoning', 'LotFrontage', 'LotArea', 'LotShape', 'LandContour',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle', 'Exterior2nd',
       'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtUnfSF',
       'TotalBsmtSF', 'HeatingQC', 'CentralAir', 'Electrical', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenQual', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageCars', 'GarageQual', 'GarageCond', 'PavedDrive',
       'WoodDeckSF', 'Fence', 'YrSold', 'SaleType', 'SaleCondition',
       'SalePrice', 'YearBuiltlabel', 'YearRemodAdd_label', 'TotalPorch'],
      dtype='object')

In [49]:
def get_encoder(test_df):
    global cate_list
    encoder = {}
    for col in test_df.columns:
        if col in cate_list:
            values = pd.unique(test_df[col].values)
            index = range(len(values))
            encoder[col] = {values[i]: index[i] for i in range(len(values))}

    return encoder

In [50]:
def get_new_df(test_df):
    global cate_list
    encoder = get_encoder(test_df)
    for col in test_df.columns:
        if col in cate_list:        
            test_df[col] = test_df[col].map(encoder[col])

    return test_df

In [51]:
cate_list = ['MSZoning', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'RoofStyle', 'Exterior2nd', 'MasVnrType', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'HeatingQC', 'CentralAir', 'Electrical', 
       'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish',
        'PavedDrive', 'Fence', 'YrSold', 'SaleType',
       'SaleCondition', 'YearBuiltlabel', 'YearRemodAdd_label','GarageQual','GarageCond' ]

In [52]:
get_encoder(full)

{'MSZoning': {'RL': 0, 'RM': 1, 'C (all)': 2, 'FV': 3, 'RH': 4},
 'LotShape': {'Reg': 0, 'IR1': 1, 'IR2': 2, 'IR3': 3},
 'LandContour': {'Lvl': 0, 'Bnk': 1, 'Low': 2, 'HLS': 3},
 'LotConfig': {'Inside': 0, 'FR2': 1, 'Corner': 2, 'CulDSac': 3, 'FR3': 4},
 'LandSlope': {'Gtl': 0, 'Mod': 1, 'Sev': 2},
 'Neighborhood': {'CollgCr': 0,
  'Veenker': 1,
  'Crawfor': 2,
  'NoRidge': 3,
  'Mitchel': 4,
  'Somerst': 5,
  'NWAmes': 6,
  'OldTown': 7,
  'BrkSide': 8,
  'Sawyer': 9,
  'NridgHt': 10,
  'NAmes': 11,
  'SawyerW': 12,
  'IDOTRR': 13,
  'MeadowV': 14,
  'Edwards': 15,
  'Timber': 16,
  'Gilbert': 17,
  'StoneBr': 18,
  'ClearCr': 19,
  'NPkVill': 20,
  'Blmngtn': 21,
  'BrDale': 22,
  'SWISU': 23,
  'Blueste': 24},
 'Condition1': {'Norm': 0,
  'Feedr': 1,
  'PosN': 2,
  'Artery': 3,
  'RRAe': 4,
  'RRNn': 5,
  'RRAn': 6,
  'PosA': 7,
  'RRNe': 8},
 'BldgType': {'1Fam': 0, '2fmCon': 1, 'Duplex': 2, 'TwnhsE': 3, 'Twnhs': 4},
 'HouseStyle': {'2Story': 0,
  '1Story': 1,
  '1.5Fin': 2,
  '1.5

In [53]:
full = get_new_df(full)

In [54]:
full.iloc[1,:]

MSZoning                   0.0
LotFrontage               80.0
LotArea                 9600.0
LotShape                   0.0
LandContour                0.0
LotConfig                  1.0
LandSlope                  0.0
Neighborhood               1.0
Condition1                 1.0
BldgType                   0.0
HouseStyle                 1.0
OverallQual                1.0
OverallCond                1.0
RoofStyle                  0.0
Exterior2nd                1.0
MasVnrType                 1.0
MasVnrArea                 0.0
ExterQual                  1.0
ExterCond                  0.0
Foundation                 1.0
BsmtQual                   0.0
BsmtCond                   0.0
BsmtExposure               1.0
BsmtFinType1               1.0
BsmtUnfSF                284.0
TotalBsmtSF             1262.0
HeatingQC                  0.0
CentralAir                 0.0
Electrical                 0.0
GrLivArea               1262.0
BsmtFullBath               0.0
BsmtHalfBath               1.0
FullBath

In [55]:
train = full[:1460]
test = full[1460:]

In [56]:
y_train = train.SalePrice
y_test = test.SalePrice
X_train = train.drop(columns=["SalePrice"])
X_test = test.drop(columns = ["SalePrice"])

In [57]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)

    
def mean_absolute_percentage_error(y_pred,y_true):
    y_true, y_pred = np.array(y_true),np.array(y_pred)
    return np.mean(np.abs((y_pred-y_true)/y_true))*100
    
from sklearn.metrics import make_scorer
mape = make_scorer(mean_absolute_percentage_error)

from sklearn.model_selection import cross_val_score
scores2 = cross_val_score(model2, X_train, y_train, scoring = mape, cv = 10)

mean = np.mean(scores2)
sd = np.std(scores2)
Model_score_2 = mean + sd
Model_score_2

NameError: name 'model2' is not defined

In [199]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()

selector = RFE(forest_reg, 1, step=1)
selector = selector.fit(X_train, y_train)

C:\Users\Liuti\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Liuti\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Liuti\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Liuti\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Liuti\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: F

C:\Users\Liuti\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Liuti\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Liuti\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Liuti\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Liuti\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: F

In [200]:
selector.ranking_
df = pd.DataFrame({"columns":X_train.columns,"rank":selector.ranking_})
var_rank = df.sort_values(by="rank")
var_rank

,columns,rank
29,GrLivArea,1
41,GarageCars,2
25,TotalBsmtSF,3
17,ExterQual,4
11,OverallQual,5
2,LotArea,6
24,BsmtUnfSF,7
35,KitchenQual,8
20,BsmtQual,9
1,LotFrontage,10


In [310]:
var_rank['columns'].values

array(['GrLivArea', 'GarageCars', 'TotalBsmtSF', 'ExterQual',
       'OverallQual', 'LotArea', 'BsmtUnfSF', 'KitchenQual', 'BsmtQual',
       'LotFrontage', 'BsmtFinType1', 'MasVnrArea', 'Neighborhood',
       'WoodDeckSF', 'TotalPorch', 'HeatingQC', 'FireplaceQu',
       'CentralAir', 'Foundation', 'BedroomAbvGr', 'Exterior2nd',
       'YearRemodAdd_label', 'LandContour', 'OverallCond', 'YrSold',
       'SaleCondition', 'YearBuiltlabel', 'BsmtExposure', 'GarageQual',
       'Fireplaces', 'FullBath', 'GarageFinish', 'HalfBath', 'RoofStyle',
       'GarageType', 'LotShape', 'MSZoning', 'BsmtFullBath', 'BldgType',
       'MasVnrType', 'LotConfig', 'ExterCond', 'Functional', 'SaleType',
       'Condition1', 'LandSlope', 'BsmtCond', 'PavedDrive', 'Fence',
       'HouseStyle', 'Electrical', 'BsmtHalfBath', 'GarageCond'],
      dtype=object)

In [204]:
var_group = []
for i in range(40):
    cols = var_rank["columns"].values
#     print(cols[:(i+20)])
    var_group.append(list(cols[:(i+10)]))

In [205]:
dummy_cols = ['MSZoning', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'BldgType',
 'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle', 'Exterior2nd', 'MasVnrType', 'ExterQual',
 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'HeatingQC',
 'CentralAir', 'Electrical', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 
 'PavedDrive', 'Fence', 'SaleType', 'SaleCondition', 'YearBuiltlabel', 'YearRemodAdd_label',"KitchenQual",'YrSold']

In [216]:
def cleaning_data(_train,_test,_cols):
    global dummy_cols
    _cols.append("SalePrice")
    _full = pd.concat([_train[_cols],_test[_cols]])
#     print(full)
    for col in _full.columns:
        if col in dummy_cols:
            _full = pd.get_dummies(_full,columns=[col])

    _train = _full[:1460]
    _test = _full[1460:]
#     print(train)
    _X_train = _train.drop(labels=["SalePrice"],axis=1)
    _X_test = _test.drop(labels=["SalePrice"],axis=1)
    _y_train = _train.SalePrice
    _y_test = _test.SalePrice

    return _X_train, _y_train, _X_test, _y_test

In [297]:
X_train_list =[]
y_train_list=[]
X_test_list=[]
y_test_list=[]

for i in range(40):
    X_train, y_train, X_test, y_test = cleaning_data(train,test,var_group[i])
    X_train_list.append(X_train)
    y_train_list.append(pd.Series(y_train.iloc[:,1]))
    X_test_list.append(X_test)
    y_test_list.append(pd.Series(y_test.iloc[:,1]))

for i in range(10):
    print("xtrain",X_train_list[i].shape)
    print("ytrain",y_train_list[i].shape)
    print("xtest",X_test_list[i].shape)
    print("ytest",y_test_list[i].shape)


xtrain (1460, 29)
ytrain (1460,)
xtest (1459, 29)
ytest (1459,)
xtrain (1460, 36)
ytrain (1460,)
xtest (1459, 36)
ytest (1459,)
xtrain (1460, 37)
ytrain (1460,)
xtest (1459, 37)
ytest (1459,)
xtrain (1460, 62)
ytrain (1460,)
xtest (1459, 62)
ytest (1459,)
xtrain (1460, 63)
ytrain (1460,)
xtest (1459, 63)
ytest (1459,)
xtrain (1460, 64)
ytrain (1460,)
xtest (1459, 64)
ytest (1459,)
xtrain (1460, 69)
ytrain (1460,)
xtest (1459, 69)
ytest (1459,)
xtrain (1460, 75)
ytrain (1460,)
xtest (1459, 75)
ytest (1459,)
xtrain (1460, 77)
ytrain (1460,)
xtest (1459, 77)
ytest (1459,)
xtrain (1460, 83)
ytrain (1460,)
xtest (1459, 83)
ytest (1459,)


In [220]:
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [221]:
lr = LinearRegression()
reg = lr.fit(X_train, y_train)
y_pred = reg.predict(X_test)
mean_absolute_percentage_error(y_test, y_pred)

11.13473522276677

In [253]:
result= {}
for i in range(4):
    X_train = X_train_list[i]
    y_train = y_train_list[i]
    X_test = X_test_list[i]
    y_test = y_test_list[i]
    from sklearn import linear_model
    lm = linear_model.LinearRegression()
    model = lm.fit(X_train,y_train)

    y_pred = lm.predict(X_test)

    from sklearn.model_selection import cross_val_score, cross_val_predict
    from sklearn import metrics

    def mean_absolute_percentage_error(y_pred,y_true):
        y_true, y_pred = np.array(y_true),np.array(y_pred)
        return np.mean(np.abs((y_pred-y_true)/y_true))*100

    from sklearn.metrics import make_scorer
    mape = make_scorer(mean_absolute_percentage_error)

    from sklearn.model_selection import cross_val_score
    scores1 = cross_val_score(model, X_train, y_train, scoring = mape, cv = 10)

    mean = np.mean(scores1)
    sd = np.std(scores1)
    Model_score_1 = mean + sd
#     print(Model_score_1)
    result[i+10]=Model_score_1

result

{10: 12.40263244953265,
 11: 12.683488521008286,
 12: 12.599417239060498,
 13: 11.627739038732155}

In [223]:
X_train, y_train, X_test, y_test = cleaning_data(train,test,var_group[i])

### Random Forest

In [29]:
result= {}
for i in range(3):
    X_train = X_train_list[i]
    y_train = y_train_list[i]
    X_test = X_test_list[i]
    y_test = y_test_list[i]
#     print(test.iloc[1,:])
#     print("y_train:",y_train)
# #     print(y_train.shape)
#     print("X_train:",X_train)


    from sklearn.ensemble import RandomForestRegressor
    rf = RandomForestRegressor(random_state = 42)
    model2 = rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    print(mean_absolute_percentage_error(y_test,y_pred))
    
    def mean_absolute_percentage_error(y_true,y_pred):
        y_true, y_pred = np.array(y_true),np.array(y_pred)
        return np.mean(np.abs((y_pred-y_true)/y_true))*100
    
    from sklearn.metrics import make_scorer
    mape = make_scorer(mean_absolute_percentage_error)
    
    from sklearn.model_selection import cross_val_score
    scores2 = cross_val_score(model2, X_train, y_train, scoring = mape, cv = 10)
    
    mean = np.mean(scores2)
    sd = np.std(scores2)
    Model_score_2 = mean + sd


    result[i+10]=Model_score_2

result

NameError: name 'X_train_list' is not defined